In [1]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat

In [2]:
from tensorflow.python.client import device_lib


from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)
print(device_lib.list_local_devices())


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15063640091308012389
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10538966172369120777
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
directory = "./data/eeg_feature_smooth/1/"
directories = ["./data/eeg_feature_smooth/{}/".format(i+1) for i in range(3)] 
print(directories)


['./data/eeg_feature_smooth/1/', './data/eeg_feature_smooth/2/', './data/eeg_feature_smooth/3/']


In [4]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 62, 5, 64)) # features = 4*62*5*64 (zero padded) // trials = (3 sessions) x 15 people x 24 labels 
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                #print(sample.shape)
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                #print(sample.shape)
                array[h][i][j][k] = sample

print(array.shape)



./data/eeg_feature_smooth/1/
./data/eeg_feature_smooth/2/
./data/eeg_feature_smooth/3/
(3, 15, 24, 4, 62, 5, 64)


In [5]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])

In [6]:
_X.shape

(1080, 4, 62, 5, 64)

In [7]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)
y.shape

(1080,)

In [106]:
X = _X.transpose(0, 4, 1, 2,3)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 62, 5)
(1080, 64, 1240)


In [36]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [43]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(
32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
])

In [127]:
def LSTM():
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
    ])

In [86]:
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [128]:
cnn = single_conv()
model = LSTM()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [129]:
model.fit(X_train, y_train,epochs=200, validation_split=0.2)

Train on 691 samples, validate on 173 samples
Epoch 1/200
691/691 [==============================] - 9s 13ms/sample - loss: 1.8958 - accuracy: 0.2981 - val_loss: 1.4696 - val_accuracy: 0.3064
Epoch 2/200
691/691 [==============================] - 7s 10ms/sample - loss: 1.3614 - accuracy: 0.3184 - val_loss: 1.3847 - val_accuracy: 0.3006
Epoch 3/200
691/691 [==============================] - 7s 10ms/sample - loss: 1.3166 - accuracy: 0.4081 - val_loss: 1.3102 - val_accuracy: 0.4393
Epoch 4/200
691/691 [==============================] - 7s 10ms/sample - loss: 1.2525 - accuracy: 0.4226 - val_loss: 1.2817 - val_accuracy: 0.3931
Epoch 5/200
691/691 [==============================] - 7s 10ms/sample - loss: 1.2582 - accuracy: 0.4110 - val_loss: 1.2553 - val_accuracy: 0.4104
Epoch 6/200
691/691 [==============================] - 7s 10ms/sample - loss: 1.2339 - accuracy: 0.4124 - val_loss: 1.2300 - val_accuracy: 0.4740
Epoch 7/200
691/691 [==============================] - 7s 10ms/sample - loss: 

691/691 [==============================] - 7s 10ms/sample - loss: 0.0826 - accuracy: 0.9783 - val_loss: 3.0864 - val_accuracy: 0.3815
Epoch 57/200
691/691 [==============================] - 7s 10ms/sample - loss: 0.0785 - accuracy: 0.9797 - val_loss: 3.2326 - val_accuracy: 0.3931
Epoch 58/200
691/691 [==============================] - 7s 10ms/sample - loss: 0.0968 - accuracy: 0.9682 - val_loss: 3.0943 - val_accuracy: 0.4220
Epoch 59/200
691/691 [==============================] - 7s 10ms/sample - loss: 0.1242 - accuracy: 0.9638 - val_loss: 2.9253 - val_accuracy: 0.4451
Epoch 60/200
691/691 [==============================] - 7s 10ms/sample - loss: 0.1108 - accuracy: 0.9696 - val_loss: 3.0933 - val_accuracy: 0.3931
Epoch 61/200
691/691 [==============================] - 7s 10ms/sample - loss: 0.1219 - accuracy: 0.9653 - val_loss: 2.9606 - val_accuracy: 0.4162
Epoch 62/200
691/691 [==============================] - 7s 10ms/sample - loss: 0.1408 - accuracy: 0.9566 - val_loss: 2.9498 - val_a

KeyboardInterrupt: 

In [22]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

216/1 - 0s - loss: 701.7454 - accuracy: 0.5602

Test accuracy: 0.5601852


In [67]:
model2 = single_conv()
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print(model2.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 62, 32)            119072    
_________________________________________________________________
flatten_14 (Flatten)         (None, 1984)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               254080    
_________________________________________________________________
dense_29 (Dense)             (None, 10)                1290      
Total params: 374,442
Trainable params: 374,442
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
model2.fit(X_train, y_train,epochs=30)

Train on 864 samples
Epoch 1/30
864/864 [==============================] - 2s 2ms/sample - loss: 150034909.7662 - accuracy: 0.2789
Epoch 2/30
864/864 [==============================] - 2s 2ms/sample - loss: 32307416.4155 - accuracy: 0.3206
Epoch 3/30
864/864 [==============================] - 2s 2ms/sample - loss: 9884428.8003 - accuracy: 0.4236
Epoch 4/30
864/864 [==============================] - 2s 2ms/sample - loss: 1419311.0480 - accuracy: 0.4190
Epoch 5/30
864/864 [==============================] - 2s 2ms/sample - loss: 321022.4887 - accuracy: 0.4491
Epoch 6/30
864/864 [==============================] - 2s 2ms/sample - loss: 298378.4297 - accuracy: 0.4884
Epoch 7/30
864/864 [==============================] - 2s 2ms/sample - loss: 422989.3785 - accuracy: 0.4780
Epoch 8/30
864/864 [==============================] - 2s 2ms/sample - loss: 411081.0179 - accuracy: 0.4549
Epoch 9/30
864/864 [==============================] - 2s 2ms/sample - loss: 1515301.1745 - accuracy: 0.4028
Epoch 10

In [69]:
test_loss, test_acc = model2.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

216/1 - 0s - loss: 43732166.9734 - accuracy: 0.6204

Test accuracy: 0.6203704


In [30]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
bestmodel = None;
bestAcc = 0;

In [31]:
cvscores = []
for train, test in kfold.split(X_train, y_train):
    model2 = single_conv()
    model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model2.fit(X_train[train], y_train[train],epochs=100)
    scores = model2.evaluate(X_train[test], y_train[test], verbose=0)
    print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))
    if(scores[1] > bestAcc):
        bestAcc = scores[1]
        bestmodel = model2
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


Train on 646 samples
Epoch 1/100
646/646 [==============================] - 2s 3ms/sample - loss: 88288571.4972 - accuracy: 0.2632
Epoch 2/100
646/646 [==============================] - 1s 2ms/sample - loss: 47619400.1416 - accuracy: 0.3204
Epoch 3/100
646/646 [==============================] - 1s 2ms/sample - loss: 1228401.5248 - accuracy: 0.3994
Epoch 4/100
646/646 [==============================] - 1s 2ms/sample - loss: 64778697.3631 - accuracy: 0.3560
Epoch 5/100
646/646 [==============================] - 1s 2ms/sample - loss: 8608718.5977 - accuracy: 0.2972
Epoch 6/100
646/646 [==============================] - 1s 2ms/sample - loss: 11311111.7831 - accuracy: 0.3467
Epoch 7/100
646/646 [==============================] - 1s 2ms/sample - loss: 4040976.1584 - accuracy: 0.4365
Epoch 8/100
646/646 [==============================] - 1s 2ms/sample - loss: 1803708.7172 - accuracy: 0.4830
Epoch 9/100
646/646 [==============================] - 1s 2ms/sample - loss: 620394.0113 - accuracy: 0.

646/646 [==============================] - 1s 2ms/sample - loss: 2066.6357 - accuracy: 0.9056
Epoch 77/100
646/646 [==============================] - 1s 2ms/sample - loss: 2033.1409 - accuracy: 0.9149
Epoch 78/100
646/646 [==============================] - 1s 2ms/sample - loss: 1611.6492 - accuracy: 0.9180
Epoch 79/100
646/646 [==============================] - 1s 2ms/sample - loss: 1401.0196 - accuracy: 0.9303
Epoch 80/100
646/646 [==============================] - 1s 2ms/sample - loss: 1760.0778 - accuracy: 0.9133
Epoch 81/100
646/646 [==============================] - 1s 2ms/sample - loss: 2636.8303 - accuracy: 0.9133
Epoch 82/100
646/646 [==============================] - 1s 2ms/sample - loss: 5709.1262 - accuracy: 0.8700
Epoch 83/100
646/646 [==============================] - 1s 2ms/sample - loss: 2839.8008 - accuracy: 0.8932
Epoch 84/100
646/646 [==============================] - 1s 2ms/sample - loss: 2919.1626 - accuracy: 0.9195
Epoch 85/100
646/646 [============================

648/648 [==============================] - 1s 2ms/sample - loss: 234.9321 - accuracy: 0.9645
Epoch 53/100
648/648 [==============================] - 1s 2ms/sample - loss: 110.7521 - accuracy: 0.9707
Epoch 54/100
648/648 [==============================] - 1s 2ms/sample - loss: 103.0205 - accuracy: 0.9753
Epoch 55/100
648/648 [==============================] - 1s 2ms/sample - loss: 137.8635 - accuracy: 0.9707
Epoch 56/100
648/648 [==============================] - 1s 2ms/sample - loss: 101.2362 - accuracy: 0.9722
Epoch 57/100
648/648 [==============================] - 1s 2ms/sample - loss: 166.2570 - accuracy: 0.9645
Epoch 58/100
648/648 [==============================] - 1s 2ms/sample - loss: 142.0960 - accuracy: 0.9660
Epoch 59/100
648/648 [==============================] - 1s 2ms/sample - loss: 255.7022 - accuracy: 0.9537
Epoch 60/100
648/648 [==============================] - 1s 2ms/sample - loss: 127.4806 - accuracy: 0.9707
Epoch 61/100
648/648 [==============================] - 1s 

649/649 [==============================] - 1s 2ms/sample - loss: 5331.0922 - accuracy: 0.8659
Epoch 29/100
649/649 [==============================] - 1s 2ms/sample - loss: 4803.7095 - accuracy: 0.8706
Epoch 30/100
649/649 [==============================] - 1s 2ms/sample - loss: 4597.0935 - accuracy: 0.8752
Epoch 31/100
649/649 [==============================] - 1s 2ms/sample - loss: 4132.9316 - accuracy: 0.8875
Epoch 32/100
649/649 [==============================] - 1s 2ms/sample - loss: 3904.0519 - accuracy: 0.8860
Epoch 33/100
649/649 [==============================] - 1s 2ms/sample - loss: 3135.7975 - accuracy: 0.8998
Epoch 34/100
649/649 [==============================] - 1s 2ms/sample - loss: 5358.8874 - accuracy: 0.8690
Epoch 35/100
649/649 [==============================] - 1s 2ms/sample - loss: 4915.2935 - accuracy: 0.8659
Epoch 36/100
649/649 [==============================] - 1s 2ms/sample - loss: 3154.4980 - accuracy: 0.8968
Epoch 37/100
649/649 [============================

649/649 [==============================] - 1s 2ms/sample - loss: 10122631.4323 - accuracy: 0.4391
Epoch 5/100
649/649 [==============================] - 1s 2ms/sample - loss: 1251971.1004 - accuracy: 0.4499
Epoch 6/100
649/649 [==============================] - 1s 2ms/sample - loss: 6805803.2188 - accuracy: 0.4422
Epoch 7/100
649/649 [==============================] - 1s 2ms/sample - loss: 237371.5536 - accuracy: 0.4946
Epoch 8/100
649/649 [==============================] - 1s 2ms/sample - loss: 71537.8727 - accuracy: 0.5039
Epoch 9/100
649/649 [==============================] - 1s 2ms/sample - loss: 59311.2466 - accuracy: 0.5378
Epoch 10/100
649/649 [==============================] - 1s 2ms/sample - loss: 51346.7578 - accuracy: 0.5470
Epoch 11/100
649/649 [==============================] - 1s 2ms/sample - loss: 46536.1900 - accuracy: 0.5501
Epoch 12/100
649/649 [==============================] - 1s 2ms/sample - loss: 41869.5791 - accuracy: 0.5824
Epoch 13/100
649/649 [================

649/649 [==============================] - 1s 2ms/sample - loss: 592.2886 - accuracy: 0.9368
Epoch 81/100
649/649 [==============================] - 1s 2ms/sample - loss: 519.8640 - accuracy: 0.9430
Epoch 82/100
649/649 [==============================] - 1s 2ms/sample - loss: 556.7180 - accuracy: 0.9430
Epoch 83/100
649/649 [==============================] - 1s 2ms/sample - loss: 561.1163 - accuracy: 0.9399
Epoch 84/100
649/649 [==============================] - 1s 2ms/sample - loss: 435.6949 - accuracy: 0.9461
Epoch 85/100
649/649 [==============================] - 1s 2ms/sample - loss: 366.1998 - accuracy: 0.9553
Epoch 86/100
649/649 [==============================] - 1s 2ms/sample - loss: 611.7861 - accuracy: 0.9414
Epoch 87/100
649/649 [==============================] - 1s 2ms/sample - loss: 379.7705 - accuracy: 0.9522
Epoch 88/100
649/649 [==============================] - 1s 2ms/sample - loss: 313.7632 - accuracy: 0.9599
Epoch 89/100
649/649 [==============================] - 1s 

In [32]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

66.07% (+/- 3.15%)


In [35]:
test_loss, test_acc = bestmodel.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

216/1 - 0s - loss: 114538685.6956 - accuracy: 0.6806

Test accuracy: 0.6805556


In [26]:
import pickle
pickle.dump( bestmodel, open( "save.p", "wb" ) )

In [34]:
test_loss, test_acc = bestmodel.evaluate(X,  y, verbose=2)
print('\nTest accuracy:', test_acc)

1080/1 - 1s - loss: 26907893.8642 - accuracy: 0.8324

Test accuracy: 0.8324074
